In [0]:
import requests
import time
import pandas as pd
import urllib3
from multiprocessing.dummy import Pool
from bs4 import BeautifulSoup as bs
from bs4 import Comment
from bs4 import Doctype

Функция, которая подключается к сайту и забирает из его html весь текст без тегов, записывая текст из title в первую переменную, из description - во вторую, а оставшийся текст - в третью.
После возвращает все три переменнные <br>
Если же подключение к сайту не было успешным - возвращает none

In [0]:
def title_parser(url):
    try:
        request = session.get(url,headers = headers,verify=False, timeout=10)
        if request.status_code == 200:
            soup = bs(request.content, 'html.parser', from_encoding='utf-8-sig')
            [c.extract() for c in soup.find_all(string=lambda text: isinstance(text, Comment))]
            [item.extract() for item in soup if isinstance(item, Doctype)]
            [tag.decompose() for tag in soup("script")]
            [tag.decompose() for tag in soup("style")]
            
            title = soup.find('title').get_text()
            [tag.decompose() for tag in soup("title")]

            try:
                meta_tag = soup.find('meta', attrs={'name': 'description'})['content']
            except:
                meta_tag = ''
            content = '.'.join(soup.findAll(text=True))
            content = [value for value in content.split() if value != '' and value != '.']
            content = ' '.join(content)
            content = ' '.join(content.split(' .'))
            
            return title, meta_tag, content
        else:
            return 'none','none','none'
    except :
        print("Ошибка " + url)
        return 'none','none','none'

Подключается датасет с url, категорией и подкатегорией сатой, поготовленный заранее <br>
Датасет делится на части по 1000 сайтов в каждой, после, используя мультитрединг, url каждой части прогоняются через вышеописанную функцию title_parser, возвращенные данные записываются в отдельные столбики и текущая часть датасета сохраняется

In [0]:
if __name__ == '__main__':
    headers = {
    'Accept' : '*/*',
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
    }

    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    session = requests.Session()

    df = pd.read_csv('/content/drive/My Drive/mailClassifier/mail_toparse_1_1.csv', encoding='utf-8-sig') 
    df = df.drop('Unnamed: 0', 1)

    dflen = len(df)
    for i in range(-(-dflen//1000)):
      if i>=0:
        start_time = time.time()
        print(i)

        df1 = df[1000*i:1000*(i+1)]
        urls = df1['url'].to_list()

        with Pool(20) as p:
            titles, desc, content = zip(*p.map(title_parser, urls))

        df1['title'] = list(titles)
        df1['description'] = list(desc)
        df1['content'] = list(content)
        
        try:
          print('Start saving...')
          if i == 0:
              with open('/content/drive/My Drive/mailClassifier/mail_parsed_1_1.csv', 'w', encoding='utf-8-sig',newline='') as f:
                  df1.to_csv(f, header=True)
          else:
              with open('/content/drive/My Drive/mailClassifier/mail_parsed_1_1.csv', 'a', encoding='utf-8-sig',newline='') as f:
                  df1.to_csv(f, header=False)
        except:
          pass

        print(f"{(time.time() - start_time)} секунд")